# SignXAI with PyTorch - ECG Time Series Explanation

This tutorial demonstrates how to use SignXAI to explain ECG (electrocardiogram) predictions with PyTorch models. We'll be working with ECG data and a PyTorch model for detecting cardiac conditions.

## Setup Requirements

For this PyTorch tutorial, you'll need to install SignXAI with PyTorch dependencies:

```bash
# For conda users
conda create -n signxai-pytorch python=3.8
conda activate signxai-pytorch
pip install -r ../../requirements/common.txt
pip install -r ../../requirements/pytorch.txt

# Or for pip users
python -m venv signxai_pytorch_env
source signxai_pytorch_env/bin/activate  # On Windows: signxai_pytorch_env\Scripts\activate
pip install -r ../../requirements/common.txt
pip install -r ../../requirements/pytorch.txt
```

## Overview

We'll cover:
1. Loading ECG data and a PyTorch model
2. Pre-processing ECG signals
3. Applying PyTorch-based explainability methods to time series data
4. Visualizing the regions of the ECG that influenced model predictions

Let's get started!

## 1. Import Libraries

In [ ]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

# SignXAI imports
from signxai.torch_signxai.methods import SIGN, GradCAM, GuidedBackprop, LRPZ, LRPEpsilon
from signxai.common.visualization import visualize_attribution
from signxai.torch_signxai.utils import remove_softmax

# Import utilities for ECG data handling
# Ensure the utils directory is in the Python path
sys.path.append(os.path.join(os.path.dirname(os.path.abspath("__file__")), "..", "utils"))
from pytorch.data import load_and_preprocess_ecg  # PyTorch-specific data loader
from tensorflow.explainability import normalize_ecg_relevancemap  # This utility can be shared

## 2. Set Up Data Paths

In [ ]:
# Set up paths
_THIS_DIR = os.path.dirname(os.path.abspath("__file__"))
DATA_DIR = os.path.realpath(os.path.join(_THIS_DIR, "..", "..", "data"))
TIMESERIES_DIR = os.path.join(DATA_DIR, "timeseries")
ECG_MODEL_PATH = os.path.join(DATA_DIR, "models", "pytorch", "ECG", "ecg_model.pt")

# Check that model file exists
assert os.path.exists(ECG_MODEL_PATH), f"ECG model not found at {ECG_MODEL_PATH}"

## 3. Define a PyTorch ECG Model

Let's first define a PyTorch model architecture for ECG processing. This is a 1D convolutional neural network designed for time series data.

In [ ]:
class ECGModel(nn.Module):
    def __init__(self, input_channels=1, sequence_length=2000):
        super(ECGModel, self).__init__()
        self.features = nn.Sequential(
            # First convolutional block
            nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(32),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # 1000
            
            # Second convolutional block
            nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=4, stride=4),  # 250
            
            # Third convolutional block
            nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2, stride=2),  # 125
            
            # Fourth convolutional block
            nn.Conv1d(128, 256, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=5, stride=5),  # 25
        )
        
        # Flatten and fully connected layers
        self.fc = nn.Sequential(
            nn.Linear(256 * 25, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 1),  # Binary classification
            nn.Sigmoid()  # Use sigmoid for binary classification
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # Flatten
        x = self.fc(x)
        return x
    
    # Convenience method to get the last convolutional layer (for GradCAM)
    def get_last_conv_layer(self):
        return self.features[9]  # The 4th Conv1d layer

## 4. Load Model and ECG Data

In [ ]:
# Load the PyTorch ECG model
try:
    ecg_model = torch.load(ECG_MODEL_PATH)
    print("Loaded pre-trained ECG model")
except Exception as e:
    print(f"Could not load saved model: {e}\nInitializing new model instead")
    # Initialize a new model if loading fails
    ecg_model = ECGModel()

# Set model to evaluation mode
ecg_model.eval()

# Remove softmax/sigmoid for explainability methods
ecg_model_no_softmax = remove_softmax(ecg_model)

# Print model architecture
print(ecg_model)

In [ ]:
# Load a sample ECG
try:
    # Try to load from saved numpy file
    ecg_sample_path = os.path.join(TIMESERIES_DIR, "ecg_sample.npy")
    ecg_data = np.load(ecg_sample_path)
    print(f"Loaded ECG sample from {ecg_sample_path}")
except Exception as e:
    print(f"Could not load ECG sample: {e}\nGenerating synthetic data instead")
    # Generate synthetic ECG data if loading fails
    # This is simplified and not realistic, just for illustration
    from scipy import signal
    t = np.linspace(0, 10, 2000)
    ecg_data = signal.square(2 * np.pi * 1.2 * t, duty=0.3) + np.sin(2 * np.pi * 3 * t)
    ecg_data += np.random.normal(0, 0.1, size=ecg_data.shape)

# Plot the ECG data
plt.figure(figsize=(15, 3))
plt.plot(ecg_data)
plt.title("ECG Sample")
plt.xlabel("Time (samples)")
plt.ylabel("Amplitude")
plt.grid(True)
plt.show()

# Preprocess for PyTorch model (add channel dimension and batch dimension)
ecg_tensor = torch.tensor(ecg_data, dtype=torch.float32).view(1, 1, -1)

## 5. Get Model Prediction

In [ ]:
# Get model prediction
with torch.no_grad():
    prediction = ecg_model(ecg_tensor)
    prediction_probability = prediction.item()

print(f"Model prediction: {prediction_probability:.4f}")
print(f"Diagnosis: {'Positive' if prediction_probability > 0.5 else 'Negative'} for abnormal ECG")

## 6. Generate Explanations for the ECG Signal

In [ ]:
# Define the explainability methods to use
methods = {
    'Gradient': SIGN(ecg_model_no_softmax),
    'Gradient × Input': SIGN(ecg_model_no_softmax),  # Will be processed separately
    'GradCAM': GradCAM(ecg_model_no_softmax, ecg_model.get_last_conv_layer()),
    'Guided Backprop': GuidedBackprop(ecg_model_no_softmax),
    'LRP-Z': LRPZ(ecg_model_no_softmax),
    'LRP-Epsilon': LRPEpsilon(ecg_model_no_softmax, epsilon=0.5)
}

# Storage for explanations
explanations = {}

In [ ]:
# Target for binary classification (in PyTorch we typically need to specify a target)
target = 0 if prediction_probability < 0.5 else 1

# Generate explanations for each method
for method_name, explainer in methods.items():
    print(f"Generating {method_name} explanation...")
    
    if method_name == 'Gradient × Input':
        # Special case for gradient × input
        grad = explainer.attribute(ecg_tensor, target=target).detach().numpy()
        explanation = grad * ecg_tensor.detach().numpy()
    else:
        explanation = explainer.attribute(ecg_tensor, target=target).detach().numpy()
    
    # Only use positive values for visualization
    explanation = np.maximum(explanation, 0)
    
    # Normalize the explanation for visualization
    explanation = normalize_ecg_relevancemap(explanation)
    
    # Add to explanations dictionary
    explanations[method_name] = explanation
    
print("All explanations generated!")

## 7. Visualize ECG Explanations

In [ ]:
# Convert explanations to proper shape for visualization
def reshape_for_visualization(explanation):
    # For 1D time series data, extract the values and flatten
    if explanation.ndim == 3:  # [batch, channel, time]
        return explanation[0, 0, :]
    elif explanation.ndim == 4:  # [batch, channel, height, width]
        # For GradCAM, the result might need to be resized
        from scipy.interpolate import interp1d
        orig_size = ecg_data.shape[0]
        expl = explanation[0, 0]
        if expl.shape[0] != orig_size:
            x_orig = np.linspace(0, 1, expl.shape[0])
            x_new = np.linspace(0, 1, orig_size)
            f = interp1d(x_orig, expl, kind='linear')
            return f(x_new)
        return expl
    return explanation

# Process explanations
processed_explanations = {}
for method_name, explanation in explanations.items():
    processed_explanations[method_name] = reshape_for_visualization(explanation)

In [ ]:
# Filter explanations for better visualization (optional)
posthresh = 0.2  # Threshold for filtering low values
cmap_adjust = 0.3  # Adjust colormap for better visibility

for method_name, explanation in processed_explanations.items():
    # Filter low values
    filtered_explanation = explanation.copy()
    filtered_explanation[filtered_explanation <= posthresh] = 0
    filtered_explanation[filtered_explanation > posthresh] += cmap_adjust
    
    # Update the explanations
    processed_explanations[method_name] = filtered_explanation

In [ ]:
# Create a figure to visualize all explanations
fig, axes = plt.subplots(len(processed_explanations), 1, figsize=(15, 4*len(processed_explanations)))

for i, (method_name, explanation) in enumerate(processed_explanations.items()):
    ax = axes[i] if len(processed_explanations) > 1 else axes
    
    # Plot the ECG
    ax.plot(ecg_data, color='blue', alpha=0.7)
    
    # Create a colormap for the explanation
    ax_twin = ax.twinx()
    ax_twin.fill_between(range(len(ecg_data)), 0, explanation, 
                         color='red', alpha=0.5, label='Explanation')
    ax_twin.set_ylim(0, 1.1)
    ax_twin.set_ylabel('Relevance', color='red')
    ax_twin.tick_params(axis='y', labelcolor='red')
    
    # Set title and labels
    ax.set_title(f"{method_name} Explanation")
    ax.set_xlabel('Time (samples)')
    ax.set_ylabel('ECG Amplitude', color='blue')
    ax.tick_params(axis='y', labelcolor='blue')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 8. Interpretation of ECG Explanations

Let's interpret what we're seeing in the ECG explanations:

- **Gradient**: Shows the sensitivity of the model output to changes in the input ECG signal. Peaks often correspond to key cardiac events (P-waves, QRS complexes, T-waves).

- **Gradient × Input**: Enhances the gradient by multiplying it with the input signal, emphasizing areas where both the signal and its importance are high.

- **GradCAM**: Adapts the Grad-CAM technique for 1D time series data, highlighting temporal regions that contributed most to the classification decision.

- **Guided Backprop**: Creates sharper feature visualizations by modifying the backpropagation signal through ReLU layers.

- **LRP-Z**: Layer-wise Relevance Propagation with the Z-rule propagates the prediction backward through the network to identify relevant input features.

- **LRP-Epsilon**: A variant of LRP that adds a stabilizing term (epsilon) to avoid division by zero, producing slightly different attribution maps.

The highlighted regions in the ECG indicate which parts of the signal were most important for the model's prediction. In cardiology, these regions often correspond to specific cardiac events like abnormal QRS complexes or ST-segment abnormalities.

## 9. Using SignXAI's TensorFlow API with PyTorch for Time Series

As in the image example, we can use SignXAI's TensorFlow-style API even with our PyTorch model.

In [ ]:
# Import the TensorFlow-style API
from signxai.torch_signxai import tf_calculate_relevancemap

# Use the TensorFlow-style API with PyTorch model
methods_tf_style = ['gradient', 'gradient_x_input', 'guided_backprop', 'lrp_z', 'lrp_epsilon_0_1']
explanations_tf_style = {}

for method in methods_tf_style:
    print(f"Generating explanation using TF-style API: {method}...")
    
    # Using the TensorFlow-style API with PyTorch model
    explanation = tf_calculate_relevancemap(method, ecg_tensor, ecg_model_no_softmax)
    
    # Process for visualization
    explanation = np.maximum(explanation, 0)  # Only use positive values
    explanation = normalize_ecg_relevancemap(explanation)
    explanation = reshape_for_visualization(explanation)
    
    # Filter low values
    explanation[explanation <= posthresh] = 0
    explanation[explanation > posthresh] += cmap_adjust
    
    # Store explanation
    explanations_tf_style[method] = explanation

In [ ]:
# Create a figure to visualize TF-style API results
fig, axes = plt.subplots(len(explanations_tf_style), 1, figsize=(15, 4*len(explanations_tf_style)))

for i, (method_name, explanation) in enumerate(explanations_tf_style.items()):
    ax = axes[i] if len(explanations_tf_style) > 1 else axes
    
    # Plot the ECG
    ax.plot(ecg_data, color='blue', alpha=0.7)
    
    # Create a colormap for the explanation
    ax_twin = ax.twinx()
    ax_twin.fill_between(range(len(ecg_data)), 0, explanation, 
                         color='green', alpha=0.5, label='Explanation')
    ax_twin.set_ylim(0, 1.1)
    ax_twin.set_ylabel('Relevance', color='green')
    ax_twin.tick_params(axis='y', labelcolor='green')
    
    # Set title and labels
    ax.set_title(f"TF-API: {method_name} Explanation")
    ax.set_xlabel('Time (samples)')
    ax.set_ylabel('ECG Amplitude', color='blue')
    ax.tick_params(axis='y', labelcolor='blue')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 10. Conclusion

In this tutorial, we've demonstrated how to use SignXAI to explain predictions of ECG models in PyTorch. We've covered:

- Creating a PyTorch model for ECG classification
- Loading and preprocessing ECG time series data
- Applying various explainability methods to ECG signals
- Visualizing and interpreting explanations
- Using SignXAI's TensorFlow-style API with PyTorch models

These techniques help medical professionals understand what parts of the ECG signal are contributing to the model's predictions, potentially enhancing trust and enabling model validation for clinical use.

The ability to seamlessly switch between PyTorch and TensorFlow APIs demonstrates SignXAI's versatility, making it a valuable tool for researchers and practitioners working with either framework.